In [3]:
from scipy.misc import comb, factorial
import numpy as np
import itertools
import pandas as pd

# Problem 1: True or false

# Problem 2

In [4]:
num_sims = 10000
num_sims_a = np.int(2/3 * num_sims)
num_sims_b = np.int(1/3 * num_sims)
num_sims = num_sims_a + num_sims_b
print("{0}, {1}, {2}".format(num_sims, num_sims_a, num_sims_b))
num_bits = 15

# source_list = np.random.choice(['a', 'b'], size=num_sims, p=[2/3, 1/3])
# df = pd.DataFrame()

# for source in source_list:
#     tmp = sim_channel(source_type=source, num_bits=num_bits)
#     tmp_df = pd.DataFrame(data=[tmp])
#     tmp_df['Source'] = source
#     df = pd.concat((df, tmp_df))

simu_a = np.random.choice([0, 1], size=(num_sims_a, num_bits), p=[0.75, 0.25])
simu_b = np.random.choice([0, 1], size=(num_sims_b, num_bits), p=[0.25, 0.75])

simu_a_df = pd.DataFrame(data=simu_a)
simu_a_df['Source'] = 'a'

simu_b_df = pd.DataFrame(data=simu_b)
simu_b_df['Source'] = 'b'

df = pd.concat((simu_a_df, simu_b_df))
df.shape

9999, 6666, 3333


(9999, 16)

In [5]:
# check if the source probabilites are correct
df.Source.value_counts(normalize=True)

a    0.666667
b    0.333333
Name: Source, dtype: float64

In [6]:
# check transition probabilites
df.groupby(by='Source')[0].value_counts(normalize=True)

Source  0
a       0    0.744974
        1    0.255026
b       1    0.755176
        0    0.244824
Name: 0, dtype: float64

## Find P(Y1=0), the probability that the first bit received is 0.

In [7]:
df[0].value_counts(normalize=True)

0    0.578258
1    0.421742
Name: 0, dtype: float64

In [8]:
3/4*2/3 + 1/4*1/3

0.5833333333333334

## Given that message A was transmitted, what is the probability that exactly 6 of the first 10 received bits are ones? (Answer with at least 3 decimal digits.)

In [9]:
df_a = df[df.Source == 'a']
df_a = df_a.loc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
df_a_sum = df_a.sum(axis=1)
df_a_sum.value_counts(normalize=True)

2    0.279628
3    0.243474
1    0.191569
4    0.150315
5    0.059706
0    0.056256
6    0.015902
7    0.003000
8    0.000150
dtype: float64

In [10]:
comb(10, 6) * (1/4)**6 * (3/4)**4

0.016222000122070312

## Find the probability that the first and second received bits are the same.

In [11]:
match12 = df.loc[:, 0] == df.loc[:, 1]
num_match12 = np.sum(match12)

print(num_match12/df.shape[0])

0.631663166317


In [12]:
(3/4 * 3/4 * 2/3) + (1/4 * 1/4 * 1/3) + (1/4 * 1/4 * 2/3) + (3/4 * 3/4 * 1/3)

0.625

## Given that Y1,…,Y5 were all equal to 0, what is the probability that Y6 is also zero?

In [13]:
tmp = df.loc[:,[0, 1, 2, 3, 4]].sum(axis=1)
df['1_to_5'] = tmp.values
use_df = df[df['1_to_5']==0]
use_df_6 = use_df[use_df.loc[:, 5] == 0]

In [14]:
use_df_6.shape[0]/use_df.shape[0]

0.7455873402312843

In [15]:
num = (3/4)**6 * (2/3) + (1/4)**6 * (1/3)
den = (3/4)**5 * (2/3) + (1/4)**5 * (1/3)
num/den

0.7489733059548254

## Find the mean of K, where K=min{i:Yi=1} is the index of the first bit that is 1.

In [16]:
# https://goo.gl/ZuCX4Q
df['first_nonzero_colnames'] = (df > 0).idxmax(axis=1, skipna=True)

# drop columns that don't have a valid count
df = df[df.first_nonzero_colnames != 'Source']
df.first_nonzero_colnames = df.first_nonzero_colnames.astype('int') + 1

In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Source,1_to_5,first_nonzero_colnames
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,a,0,10
1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,a,1,4
2,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,a,2,1
3,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,a,2,4
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,a,0,6


In [18]:
#df.first_nonzero_colnames.value_counts()
np.mean(df.first_nonzero_colnames.astype('int'))

2.982436660946805

In [19]:
#12/5

#p = 2/3*1/4 + 1/3*3/4
#1/p
2/3*4 + 1/3*4/3


3.1111111111111107

## Part 2

In [20]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Source,1_to_5,first_nonzero_colnames
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,a,0,10
1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,a,1,4
2,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,a,2,1
3,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,a,2,4
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,a,0,6


In [21]:
df['Z_sum'] = df.loc[:, 4] + df.loc[:, 5]
df['Z_diff'] = df.loc[:, 4] - df.loc[:, 5]

In [23]:
pd.crosstab(df[0], df['Z_sum'])/num_sims

Z_sum,0,1,2
0,,,
0,0.272227,0.216622,0.080208
1,0.117312,0.156316,0.148115


**Theory**

In [1]:
(3/4)**3*(2/3) + (1/4)**3*(1/3)

0.2864583333333333

In [25]:
prod_a = (3/4)*(2/3) + (1/4)*(1/3)
prod_b = (3/4)**2*(2/3) + (1/4)**2*(1/3)
prod_a * prod_b

0.2309027777777778

In [26]:
#pd.crosstab(df[0], df['Z_diff']).apply(lambda r: r/r.sum(), axis=1)
pd.crosstab(df[0], df['Z_diff'])/num_sims

Z_diff,-1,0,1
0,,,
0,0.109511,0.352435,0.107111
1,0.077908,0.265427,0.078408


In [30]:
prob_joint = (3/4)**3 * (2/3) + (1/4)**3 * (1/3) + (3/4 * 1/4 * 1/4) * (2/3) + (1/4 * 3/4 * 3/4) * (1/3)
prob_marg_a = (3/4)*(2/3) + (1/4)*(1/3)
prob_marg_b = (3/4 * 3/4) * (2/3) + (1/4 * 1/4) * (1/3) + (1/4 * 1/4) * (2/3) + (3/4 * 3/4 ) * (1/3)
print("{0}, {1}".format(prob_joint, prob_marg_a*prob_marg_b))

0.3645833333333333, 0.36458333333333337


# Problem 3: Six-sided die

## Find the probability that the results of the 6 rolls are all different. (Answer with at least 3 decimal digits.)


In [53]:
num = factorial(6)
den = 6**6
num/den

0.015432098765432098

In [54]:
#print(list(itertools.permutations([1,2,3])))
#print(list(itertools.product([1,2,3], [1,2, 3], [1, 2, 3])))
l = range(1, 7)
#all_l = list(itertools.product(l, l, l))
all_l = list(itertools.product(l, l, l, l, l, l))
n = len(all_l)
n_unique = []

num_twos = 0
num_threes = 0
for tmp in all_l:
    n_unique.append(len(set(tmp)))

#print('full:{0}, 2:{1}, 3:{2}'.format(n, num_twos, num_threes))
print(set(n_unique))
print(np.bincount(n_unique))

{1, 2, 3, 4, 5, 6}
[    0     6   930 10800 23400 10800   720]


In [123]:
(6 + 15 + 20 + 15 + 6) * comb(6,2)

930.0

In [128]:
(30 + 60 + 60 + 30 + 60 + 90 + 60 + 60 + 60 + 30) * comb(6, 3)

10800.0

In [55]:
930/n

0.01993312757201646

In [56]:
10800/n

0.23148148148148148